In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from weight_discovery import *
from free_diffusion import *

In [ ]:
from kyle_tools import rc_dict

rc_dict['legend.loc'] = 'upper right'
rc_dict['legend.fontsize'] = 12

for k,v in rc_dict.items():
    plt.rcParams[k] = v



In [ ]:
init_params = [.2,.3,.025]
#init_params = [1,.1,.1]
#ini_params = [1,.6,.1]
#plt.plot(realepr(np.linspace(0,.25,1000),*init_params))
params['init'] = init_params

In [ ]:
#Parameters
#sim params
T = .25# Total time
dt = 0.001
num_steps = int(T / dt)
num_paths = 500_000

sim_params = [num_paths, num_steps, dt]

params['sim'] = sim_params

#this will skip steps when saving the output to save memory
skip = 1


In [ ]:
# Simulate the process
process = simulate_free_diffusion_underdamped(params, save_skip=skip)
process.shape


In [ ]:
num_est = 6
time_step = 20

proc_dt = dt * skip

ep = get_step_epr(process, time_step, proc_dt, params['gamma'], num_est)



In [ ]:
#steps = [10, 50, 100, 200, 300, 400, 485 ]
steps = [ int(5*i) for i in  [1, 2, 4.2, 10, 15, 20, 25, 35, 47]]
eps=[]
for time_step in steps:
    eps.append(get_step_epr(process, time_step, proc_dt, params['gamma'], num_est))


In [ ]:
fig, ax = plt.subplots()
ax.plot(np.arange(0,T,proc_dt),theo, c='k')


ax.scatter([s*proc_dt for s in steps], eps, marker='D')

In [ ]:
num_iter = 100

num_ests = [4, 6, 9, 10]
steps = [1, 3, 5, 15, 30, 100]
all_estimates = []

for i in range(num_iter):
    process = simulate_free_diffusion_underdamped(num_paths, num_steps, dt, *params['init'], save_skip=skip)
    print(f'sim {i} done', end='\r')

    estimates = []
    for num_est in num_ests:
        n_resolved_estimates=[]
        for time_step in steps:
            ep = get_step_epr(process, time_step, proc_dt, gamma, num_est)
            n_resolved_estimates.append(ep)
        estimates.append(n_resolved_estimates)
        print(f'finished {i} run of {num_est} estimates', end='\r')
    
    all_estimates.append(estimates)


In [ ]:
flst = kt.file_list('data/free_diff/mats/E10/')
flst.sort()
flst


In [ ]:
flst = flst[2:3]
flst

In [ ]:
#import data
#flst = kt.file_list('24_03_14/')
#flst = kt.file_list('data/free_diff/')



#data_list = [[kt.open_json(item) for item in flst ][0]]
data_list = [kt.open_json(item) for item in flst ]
skip=1






In [ ]:
data_list[0]['params']['sim'][0]

In [ ]:
data_list[0]['params']

In [ ]:
idxs = range(len(data_list))
percent = .15
pwr = 2

num_path = []
data = []
reg_datas = []

for idx in idxs:
    print(idx)

    #pt = 1E13*np.percentile(data_list[idx]['Xis'], percent, axis=(-1,-2))[:,:,:20,None,None] * np.identity(data_list[idx]['Xis'].shape[-1])[None,None,None,:,:] / (data_list[idx]['params']['sim'][0])**pwr
    pt =  np.identity(data_list[idx]['Xis'].shape[-1])[None,None,None,:,:] / (data_list[idx]['params']['sim'][0])**pwr

    reg_X = pt + data_list[idx]['Xis']

    reg_data = np.empty(data_list[idx]['data'].shape)

    for i in range(reg_X.shape[0]):
        for j in range(reg_X.shape[1]):
            for k in range(reg_X.shape[2]):
                X = reg_X[i,j,k]
                mu = data_list[idx]['mus'][i,j,k,...]
                reg_data[i,j,k] = (mu @ np.linalg.inv(X) @ mu) / data_list[idx]['params']['sim'][-1]

    data_list[idx]['reg_data'] = reg_data
    data_list[idx]['reg_Xis'] = reg_X

    num_path.append(data_list[idx]['params']['sim'][0])
    data.append(data_list[idx]['data'])
    reg_datas.append(reg_data)



In [ ]:
output_dict = {}
keys = list(data_list[idx].keys())
print(keys)
for i,item in enumerate(keys):
    if 'Xis' in item:
        del(keys[i])

for k in keys:
    output_dict[k] = data_list[idx][k]

output_dict['data'] = data
output_dict['num_path'] = num_path
output_dict['reg_data'] = reg_datas

output_dict['reg_param'] = 'pwr=2'


In [ ]:
kt.save_as_json(output_dict, dir='./data/free_diff/mats/', name='OS_rate_reg_E06')

In [ ]:
all_estimates.shape

In [ ]:

#datakey = 'data'
datakey = 'reg_data'



data = [ d[datakey] for d in data_list]

all_estimates = np.array(data)

#print([d.shape for d in data])
#all_estimates = (np.vstack(data)/skip**2)




dt = data_list[0]['params']['sim'][-1]
steps = data_list[0]['steps']
num_ests =  data_list[0]['num_ests']
num_iter = len(all_estimates)
num_paths = data_list[0]['params']['sim'][0]
params = data_list[0]['params']
times =  data_list[0]['t']

all_e = all_estimates

means = all_e.mean(axis=1).squeeze()
stds = all_e.std(axis=1).squeeze()


In [ ]:
num_path

In [ ]:
means.shape

In [ ]:
plt.plot(times, theo, c='k', linestyle='--', zorder=10000)
plt.errorbar(times[:-2], means, yerr=stds);

In [ ]:
pmeans.shape

In [ ]:
'''
keys = ['estimates', 'N', 'p0,Sp0,Sx0', 'dt', 'proc_dt', 'num_est', 'steps', 'gamma']
vals = [all_e, num_paths, ini_params, dt, proc_dt, num_ests, steps, gamma]

save_dict = {k:v for k,v in zip(keys, vals)}
kt.save_as_json(save_dict, 'free_diff_monotonic')
'''

In [ ]:
len(all_e[0])

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(7.5,4.75), width_ratios=[1.5,1])
proc_dt = dt * skip

plot_skip =3

theo = realepr(times,*params['init'])





for i in range(means.shape[0]):
    if i in [0,2,4]:
        num_paths = num_path[i]
        dot_args = {'markersize':6, 'marker':'.', 'linestyle':'none', 'alpha':.7, }
        #ax[0].errorbar([s*proc_dt for s in steps], pmeans[i,:], yerr=3*pstds[i,:]/np.sqrt(len(all_e)), markersize=4, marker='D', label=f'{pnum_ests[i]**2} estimates', linestyle='none', alpha=.4 )
        ax[0].errorbar([s*proc_dt for s in steps][::plot_skip], means[i,::plot_skip], yerr=3*stds[i,::plot_skip]/np.sqrt(len(all_e)), label=f'$N=${num_paths}', **dot_args )

        ax[1].errorbar([proc_dt*(s+i/6) for s in steps][::plot_skip], (means[i,::plot_skip]-theo[steps][::plot_skip])/theo[steps][::plot_skip], yerr=3*stds[i,::plot_skip]/(np.sqrt(len(all_e[i]))*theo[steps][::plot_skip]),**dot_args)
        #ax[1].errorbar([proc_dt*(s+i/6) for s in steps], (means[i,:]-theo[steps]), yerr=3*stds[i,:]/np.sqrt(num_iter), marker='D', linestyle='none' )


ax[0].plot(times, theo, c='k', linestyle='--', zorder=10000, label='$\Sigma$')
#fig.legend()

#for i, e in zip(idxs,estimates):
#    ax.scatter(i, e , color='r', marker='D')

#ax[0].set_yscale('log')
#ax[1].set_xscale('log')
#fig.suptitle(f"N={num_paths}, dt={proc_dt}, params={params['init']}, n_iteration={len(all_e)}")
#fig.suptitle(f"{pnum_ests[0]**2} estimators, dt={proc_dt}, params={params['init']}, n_iteration={len(all_e)}")

#ax[1].axhline(0, c='k')
ax[0].set_xlabel('t')

ax[1].set_ylim([-.1,.1])
ax[0].set_ylim([5,15])


In [ ]:

fig.legends = []

In [ ]:
fig.legend()
#fig.legends = []

ax[1].set_ylim([-.05,.1])
ax[0].set_ylim([5.5,10])
ax[1].axhline(0, c='k', linestyle='--', zorder=100)
ax[0].set_ylabel('$\hat{\Sigma}$')
ax[1].set_xlabel('t')
ax[1].set_ylabel('$\hat{\Sigma}$ relative error')
fig

In [ ]:
EPR_scaling_10est_20iter_250steps_10_100_1000.pdf

In [ ]:
fig.savefig('EPR_scaling_10est_20iter_250steps_10_100_1000.pdf')

In [ ]:
EP.shape

In [ ]:
EP_theo[ssteps]

In [ ]:
all_e[...,:-1][...,::2].shape

In [ ]:
np.linspace(0,1,10)[0:-1::2]

In [ ]:
len(ssteps)

In [ ]:
EP.shape

In [ ]:
#fig, ax = plt.subplots(1,2, figsize=(15,5))

EP_skip = 1
ssteps = steps[::EP_skip][:-1]
print(len(ssteps))

EP = np.cumsum( (np.diff(times[steps[::EP_skip]]))*(all_e[...,::EP_skip][:,:,:-1]), axis=-1)[...,:]
EP_M = np.mean(EP, axis=0)
EP_S = 3*np.std(EP, axis=0)/(np.sqrt(EP.shape[0])) 
EP_theo = proc_dt* np.cumsum(theo)

#ax[0].plot(times, EP_theo, c='k')

for item in EP[:,0,:]:
    ax[0].plot([s*proc_dt for s in ssteps], item-EP_theo[ssteps], alpha=.5)


for m,s,n in zip (EP_M, EP_S, num_ests):
    print( (m-EP_theo[ssteps]).shape, s.shape, n,)
    ax[1].errorbar([s*proc_dt for s in ssteps], (m-EP_theo[ssteps])/EP_theo[ssteps], yerr=s/EP_theo[ssteps], marker='o', label=f'{n}', alpha=.5 );

fig.legend()
ax[1].axhline(0, c='k')
#ax.set_xlim(.1,.25)
#ax[1].set_ylim(-.02,.1)

In [ ]:
ax[1].set_ylim(-.02,.1)
fig

In [ ]:
fig, ax = plt.subplots(1,2)

ssteps = steps[:-1]

EP = np.cumsum((np.diff(times[steps]))*(all_e[...,:-1]), axis=-1)[:,:-2]
EP_M = np.mean(EP, axis=0)
EP_S = 3*np.std(EP, axis=0)/(np.sqrt(EP.shape[0])) 
EP_theo = proc_dt* np.cumsum(theo)

#ax[0].plot(times, EP_theo, c='k')

for item in EP[:,-1,:]:
    ax[0].plot([s*proc_dt for s in ssteps], item-EP_theo[ssteps], alpha=.5)


for m,s,n in zip (EP_M, EP_S, num_ests):
    print( (m-EP_theo[ssteps]).shape, s.shape, n,)
    ax[1].errorbar([s*proc_dt for s in ssteps], (m-EP_theo[ssteps])/EP_theo[ssteps], yerr=s/EP_theo[ssteps], marker='o', label=f'{n}', alpha=.5 );

fig.legend()
ax[1].axhline(0, c='k')
#ax.set_xlim(.1,.25)
ax[1].set_ylim(-.01,.01)

In [ ]:
fig, ax = plt.subplots(1,2)

ssteps = steps[:-1]

EP = np.cumsum((np.diff(times[steps]))*(all_e[...,:-1]), axis=-1)[:,:-2]
EP_M = np.mean(EP, axis=0)
EP_S = 3*np.std(EP, axis=0)/(np.sqrt(EP.shape[0])) 
EP_theo = proc_dt* np.cumsum(theo)

#ax[0].plot(times, EP_theo, c='k')

for item in EP[:,-1,:]:
    ax[0].plot([s*proc_dt for s in ssteps], item-EP_theo[ssteps], alpha=.5)


for m,s,n in zip (EP_M, EP_S, num_ests):
    print( (m-EP_theo[ssteps]).shape, s.shape, n,)
    ax[1].errorbar([s*proc_dt for s in ssteps], (m-EP_theo[ssteps])/EP_theo[ssteps], yerr=s/EP_theo[ssteps], marker='o', label=f'{n}', alpha=.5 );

fig.legend()
ax[1].axhline(0, c='k')
#ax.set_xlim(.1,.25)
ax[1].set_ylim(-.01,.01)

In [ ]:
EP[22,:,-1]

In [ ]:
EM = np.argmax(all_e, axis=0)
np.argmax(EM, axis=0)

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(15,10))

theo = realepr(np.arange(0,T,proc_dt),*ini_params )



for i in range(0,4):
    ax[0].plot(np.arange(0,T,proc_dt),theo, c='k', linestyle='--')
    ax[0].errorbar([s*proc_dt for s in steps], means[i,:], yerr=3*stds[i,:]/np.sqrt(num_iter), marker='D', label=f'{num_ests[i]} estimators', linestyle='none' )

    ax[1].errorbar([proc_dt*(s+i/6) for s in steps], (means[i,:]-theo[steps])/theo[steps], yerr=3*stds[i,:]/np.sqrt(num_iter)/theo[steps], marker='D', linestyle='none' )


fig.legend()

#for i, e in zip(idxs,estimates):
#    ax.scatter(i, e , color='r', marker='D')

#ax[0].set_yscale('log')
ax[1].set_xscale('log')
fig.suptitle(f'N={num_paths}, dt={proc_dt}, params={ini_params}, n_iteration={num_iter}')

ax[1].axhline(0, c='k')
ax[0].set_xlabel('t')
ax[0].set_ylabel('entropy rate')
ax[1].set_xlabel('time step')
ax[1].set_ylabel('percent error')



In [ ]:
fig.savefig('ep_estimation_nonmonoE10.pdf')